In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

from datetime import datetime, timedelta

In [2]:
zomato = pd.read_csv('./data/zomato_preprocessed_3.csv')
zomato = zomato.assign(order_time=pd.to_datetime(zomato['order_time']))
zomato

,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,is_affordable_voucher,is_affordable,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected
0,543991243,2021-09-01 00:00:51,ygyH_eaSHCTSSgAda33qDLYA,False,True,NaN,NaN,NaN,TGO,67.43,...,False,True,13.486,Chennai,53.944,False,20.0,False,5.44,False
1,543992115,2021-09-01 00:02:50,vagdCBP6L9S461DAPk0tNs-A,False,True,NaN,NaN,NaN,TGO,53.50,...,False,True,NaN,Mumbai,53.500,True,NaN,False,9.17,False
2,543992763,2021-09-01 00:04:26,65RUZmnykIRzyWaqnOaB_EdQ,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,72.99,...,False,False,NaN,Chennai,72.990,True,NaN,False,6.92,False
3,543993755,2021-09-01 00:06:41,THNssTsOurRqOJf9sQEhwiRg,False,True,NaN,NaN,NaN,TGO,206.00,...,False,False,NaN,Chennai,206.000,True,NaN,False,5.83,False
4,543994010,2021-09-01 00:07:13,OGX49Bsq0pR5qB32sT83JhSg,False,True,NaN,NaN,NaN,TGO,100.99,...,False,False,NaN,Chennai,100.990,True,NaN,False,7.41,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118275,644315564,2022-01-02 23:57:08,xdSbyNHiYORwiK2AHYieL5UQ,False,False,CUSTOMER_MISTAKE_ERROR,Changed mind - Mistake / Error,CUSTOMER,TGO,147.49,...,False,False,NaN,Chennai,147.490,True,NaN,False,6.62,False
118276,644315682,2022-01-02 23:57:25,AYeFPGmOUyT4OIWfmTSCYikw,False,True,NaN,NaN,NaN,TGO,110.49,...,False,False,NaN,Coimbatore,110.490,True,NaN,False,9.90,False
118277,644316048,2022-01-02 23:58:05,COnO0tda5iSTa-t5dFFVTpwg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,112.49,...,False,False,NaN,Chennai,112.490,True,NaN,False,7.36,False
118278,644316064,2022-01-02 23:58:07,iI6rRPz2LkRi2Ct0zTaksrOA,False,True,NaN,NaN,NaN,TGO,739.24,...,False,False,NaN,Chennai,739.240,True,NaN,False,5.17,False


# Quantity

In [3]:
zomato = zomato.assign(portion=(zomato['basket_amount_lc']/75).round())
zomato.loc[zomato['portion'].round()==0, 'portion'] = 1
display(zomato['portion'].describe())

zomato = zomato.assign(portion=zomato['portion'].astype(int))

count    118280.000000
mean          1.854616
std           1.611278
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          95.000000
Name: portion, dtype: float64

# Satisfaction

In [4]:
zomato_success = zomato[zomato['is_successful']==True]
zomato_failed = zomato[zomato['is_successful']==False]

zomato_gold_success = zomato_success[zomato_success['is_gold']==True].reset_index(drop=True)
zomato_regular_success = zomato_success[zomato_success['is_gold']==False].reset_index(drop=True)

zomato_fail_ressurated = zomato_failed[zomato_failed['is_ressurected']==True].reset_index(drop=True)
zomato_fail_died = zomato_failed[zomato_failed['is_ressurected']==False].reset_index(drop=True)

NO_COURIER = zomato_fail_died[zomato_fail_died['reason']=='NO_COURIER'].reset_index(drop=True)
ELSE_REASON = zomato_fail_died[~zomato_fail_died['reason'].isin(['LATE_DELIVERY', 'NO_COURIER'])].reset_index(drop=True)

LATE_DELIVERY = zomato_fail_died[zomato_fail_died['reason']=='LATE_DELIVERY']

# 진짜 늦은 애랑, 진짜 늦지는 않은 애
REAL_LATE_DELIVERY = LATE_DELIVERY[LATE_DELIVERY['order_delay'] > 0].reset_index(drop=True)
NOT_REAL_LATE_DELIVERY = LATE_DELIVERY[LATE_DELIVERY['order_delay'] <= 0].reset_index(drop=True)

# 얘는 없애자
NULL_DELAY_LATE_DELIVERY = LATE_DELIVERY[LATE_DELIVERY['order_delay'].isnull()].reset_index(drop=True)

In [5]:
print(LATE_DELIVERY.shape[0])
print(REAL_LATE_DELIVERY.shape[0])
print(NOT_REAL_LATE_DELIVERY.shape[0])
print(NULL_DELAY_LATE_DELIVERY.shape[0])

369
91
198
80


In [20]:
91 / (91+198) * 100

31.4878892733564

In [6]:
satisfaction_score = [1, 2, 3, 4, 5]

# NO_COURIER, LATE_DELIVERY 중에서 실제로 늦은 거는 bad reason
# 그 외의 주문에는 notbad reason
bad_reason_faild_weight = [0.38, 0.52, 0.1, 0, 0]
notbad_reason_faild_weight = [0.3, 0.52, 0.18, 0, 0]

# 성공 주문의 경우 gold와 regular의 경우를 구분
gold_success_weight = [0.05, 0.12, 0.28, 0.39, 0.16]
regular_success_weight = [0.04, 0.11, 0.28, 0.4, 0.19]

In [7]:
print(sum(bad_reason_faild_weight))
print(sum(notbad_reason_faild_weight))

print(sum(gold_success_weight))
print(sum(regular_success_weight))

1.0
1.0
1.0
1.02


## Successed

In [8]:
# gold success
for i in tqdm(range(zomato_gold_success.shape[0])) : 
    zomato_gold_success.loc[zomato_gold_success.index==i, 'satisfaction'] = satisfaction_score[random.choices(range(0, 5), weights=gold_success_weight)[0]]
zomato_gold_success = zomato_gold_success.assign(satisfaction=zomato_gold_success['satisfaction'].astype(int))
zomato_gold_success

100%|██████████████████████████████████████████████████████████████████████████| 11291/11291 [00:02<00:00, 5012.09it/s]


,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,544311540,2021-09-01 14:52:45,2UwQnNjWHuT4WCiQjqU3NXEA,False,True,NaN,NaN,NaN,TGO,214.99,...,75.0,Chennai,214.99,True,25.86,True,NaN,False,4,3
1,544319257,2021-09-01 15:03:41,Xz8c3G8OL1SjOJac8nG8oIBg,False,True,NaN,NaN,NaN,TGO,150.00,...,75.0,Manali,150.00,True,33.33,True,NaN,False,3,4
2,544328121,2021-09-01 15:16:06,4-G8YphWitQ1WGEBhDQX7hVA,False,True,NaN,NaN,NaN,TGO,117.99,...,50.0,Chennai,117.99,True,29.76,True,NaN,False,2,4
3,544337601,2021-09-01 15:29:24,sygS7rYWPWTiWlJbY6TNA_1A,False,True,NaN,NaN,NaN,TGO,369.25,...,100.0,Chennai,369.25,True,21.31,True,NaN,False,6,3
4,544650932,2021-09-01 21:08:27,k6l2wtgeGBQieO6pgHc2SUfw,False,True,NaN,NaN,NaN,TGO,241.99,...,100.0,Delhi,241.99,True,29.24,True,NaN,False,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11286,644302413,2022-01-02 23:30:13,weqmoYLqteQ8qOq_LXKnpLcw,False,True,NaN,NaN,NaN,TGO,197.49,...,NaN,Chennai,197.49,True,NaN,True,NaN,False,3,4
11287,644302615,2022-01-02 23:30:39,zMdSeVGGvOSaGOEsGuuh0a5Q,False,True,NaN,NaN,NaN,TMP,505.00,...,NaN,Chennai,505.00,True,NaN,True,NaN,False,6,2
11288,644304210,2022-01-02 23:33:51,spUunm1CSpRuCuVI1KUevbfg,False,True,NaN,NaN,NaN,TGO,89.99,...,NaN,Mysore,89.99,True,NaN,True,NaN,False,1,5
11289,644307249,2022-01-02 23:39:57,4X6t0M7aZkQGaooy72SktXfg,False,True,NaN,NaN,NaN,TGO,255.44,...,NaN,Chennai,255.44,True,NaN,True,NaN,False,3,4


In [9]:
# regular success
for i in tqdm(range(zomato_regular_success.shape[0])) : 
    zomato_regular_success.loc[zomato_regular_success.index==i, 'satisfaction'] = satisfaction_score[random.choices(range(0, 5), weights=regular_success_weight)[0]]
zomato_regular_success = zomato_regular_success.assign(satisfaction=zomato_regular_success['satisfaction'].astype(int))
zomato_regular_success

100%|██████████████████████████████████████████████████████████████████████████| 87274/87274 [00:27<00:00, 3190.97it/s]


,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,543991243,2021-09-01 00:00:51,ygyH_eaSHCTSSgAda33qDLYA,False,True,NaN,NaN,NaN,TGO,67.43,...,13.486,Chennai,53.944,False,20.0,False,5.44,False,1,4
1,543992115,2021-09-01 00:02:50,vagdCBP6L9S461DAPk0tNs-A,False,True,NaN,NaN,NaN,TGO,53.50,...,NaN,Mumbai,53.500,True,NaN,False,9.17,False,1,4
2,543993755,2021-09-01 00:06:41,THNssTsOurRqOJf9sQEhwiRg,False,True,NaN,NaN,NaN,TGO,206.00,...,NaN,Chennai,206.000,True,NaN,False,5.83,False,3,5
3,543994010,2021-09-01 00:07:13,OGX49Bsq0pR5qB32sT83JhSg,False,True,NaN,NaN,NaN,TGO,100.99,...,NaN,Chennai,100.990,True,NaN,False,7.41,False,1,3
4,543995285,2021-09-01 00:10:21,NiPNR-NifZRviafzNw9WMEkg,False,True,NaN,NaN,NaN,TGO,228.11,...,NaN,Delhi,228.110,True,NaN,False,6.99,False,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87269,644314646,2022-01-02 23:55:06,KInsM3IGwvTbGucAIBmfUOBQ,False,True,NaN,NaN,NaN,TGO,269.73,...,NaN,Chennai,269.730,True,NaN,False,9.96,False,3,4
87270,644314669,2022-01-02 23:55:09,RyZAMq2ec2TpeHLJ21bSCyiA,False,True,NaN,NaN,NaN,TGO,141.99,...,NaN,Chennai,141.990,True,NaN,False,5.42,False,2,2
87271,644315682,2022-01-02 23:57:25,AYeFPGmOUyT4OIWfmTSCYikw,False,True,NaN,NaN,NaN,TGO,110.49,...,NaN,Coimbatore,110.490,True,NaN,False,9.90,False,1,5
87272,644316064,2022-01-02 23:58:07,iI6rRPz2LkRi2Ct0zTaksrOA,False,True,NaN,NaN,NaN,TGO,739.24,...,NaN,Chennai,739.240,True,NaN,False,5.17,False,10,2


## Failed

In [10]:
# notbad_ELSE_REASON
for i in tqdm(range(ELSE_REASON.shape[0])) : 
    ELSE_REASON.loc[ELSE_REASON.index==i, 'satisfaction'] = satisfaction_score[random.choices(range(0, 5), weights=notbad_reason_faild_weight)[0]]
ELSE_REASON = ELSE_REASON.assign(satisfaction=ELSE_REASON['satisfaction'].astype(int))
ELSE_REASON

100%|██████████████████████████████████████████████████████████████████████████| 12486/12486 [00:02<00:00, 4903.32it/s]


,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,544004984,2021-09-01 00:34:44,UINGQc1SwQSASus_1Q2G_Mdg,False,False,VENDOR_NO_RESPONSE,NaN,VENDOR,TGO,102.990,...,NaN,Chennai,102.990,True,NaN,False,7.51,False,1,2
1,544006330,2021-09-01 00:38:25,EwfDkQOW0LRRW65sO2lgEGtQ,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TMP,69.996,...,NaN,Chennai,69.996,True,NaN,False,9.20,False,1,2
2,544020375,2021-09-01 01:23:17,JKkWUDMuB-TGugyfM5ZRaDBA,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TGO,249.000,...,NaN,Delhi,249.000,True,NaN,False,5.76,False,3,3
3,544028990,2021-09-01 01:59:43,uyloSNjql7SxqWsrjMha0wgQ,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TMP,160.000,...,NaN,Chennai,160.000,True,NaN,False,6.76,False,2,2
4,544034443,2021-09-01 02:29:57,iTjnN6CCYiTkCtoFCGkUh9Tg,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TGO,179.990,...,NaN,Delhi,179.990,True,NaN,False,7.09,False,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,644295830,2022-01-02 23:18:32,eChVFPwq48SmqHT2wkng9bpg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,45.940,...,NaN,Chennai,45.940,True,NaN,False,5.02,False,1,1
12482,644299309,2022-01-02 23:24:39,T3R-gTCSbiS2SPlJCXbrs_HA,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TMP,183.420,...,NaN,Chennai,223.420,False,NaN,False,6.54,False,3,2
12483,644301918,2022-01-02 23:29:20,C4-ATkc2O0Tv2gVxcwKSc_6w,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TMP,128.000,...,NaN,Chennai,128.000,True,NaN,True,NaN,False,1,3
12484,644315564,2022-01-02 23:57:08,xdSbyNHiYORwiK2AHYieL5UQ,False,False,CUSTOMER_MISTAKE_ERROR,Changed mind - Mistake / Error,CUSTOMER,TGO,147.490,...,NaN,Chennai,147.490,True,NaN,False,6.62,False,2,1


In [11]:
# notbad_reason : NOT_REAL_LATE_DELIVERY
for i in tqdm(range(NOT_REAL_LATE_DELIVERY.shape[0])) : 
    NOT_REAL_LATE_DELIVERY.loc[NOT_REAL_LATE_DELIVERY.index==i, 'satisfaction'] = satisfaction_score[random.choices(range(0, 5), weights=notbad_reason_faild_weight)[0]]
NOT_REAL_LATE_DELIVERY = NOT_REAL_LATE_DELIVERY.assign(satisfaction=NOT_REAL_LATE_DELIVERY['satisfaction'].astype(int))
NOT_REAL_LATE_DELIVERY

100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 4833.30it/s]


,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,544261707,2021-09-01 13:40:48,md0lT2NW1aQ9Ww6ZNiS91QvQ,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,35.9900,...,NaN,Chennai,35.9900,True,NaN,False,7.56,False,1,1
1,544502189,2021-09-01 18:41:08,LvMyfI-WHzTIWm7f-0KVscoQ,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,102.9648,...,NaN,Chennai,102.9648,True,NaN,False,9.37,False,1,2
2,545015890,2021-09-02 13:11:09,OdmlAC8e6lSMeJDX8jAknaZg,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,88.9700,...,NaN,Chennai,88.9700,True,NaN,False,7.48,False,1,2
3,545140689,2021-09-02 15:56:28,7xXdPBvy6-RXyGx3v7zyaSCA,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,511.9900,...,NaN,Chennai,511.9900,True,NaN,False,9.18,False,7,1
4,545537423,2021-09-02 22:41:51,K33iC1bus0QnuFvCb0qZZBoA,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,159.9900,...,NaN,Vadodara,159.9900,True,NaN,False,6.57,False,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,642649230,2021-12-31 23:57:55,rPGYmwMWoDTdW_TGMbJp1oMg,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,75.8500,...,NaN,Chennai,75.8500,True,NaN,False,5.29,False,1,2
194,642668819,2022-01-01 00:37:40,aiVOBr2qO_TBq1Ma22MFSXvA,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,186.9900,...,NaN,Delhi,186.9900,True,NaN,False,5.21,False,2,1
195,642712708,2022-01-01 02:50:34,TI6I_ndKjCSrK1cJdg2kkA_A,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,141.4900,...,NaN,Chennai,141.4900,True,NaN,False,6.07,False,2,1
196,643381453,2022-01-01 21:40:22,Sp4950e-yVSR-EwveWTYEzCQ,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,315.0300,...,NaN,Chennai,315.0300,True,NaN,False,9.79,False,4,1


In [12]:
# bad_reason : REAL_LATE_DELIVERY
for i in tqdm(range(REAL_LATE_DELIVERY.shape[0])) : 
    REAL_LATE_DELIVERY.loc[REAL_LATE_DELIVERY.index==i, 'satisfaction'] = satisfaction_score[random.choices(range(0, 5), weights=bad_reason_faild_weight)[0]]
REAL_LATE_DELIVERY = REAL_LATE_DELIVERY.assign(satisfaction=REAL_LATE_DELIVERY['satisfaction'].astype(int))
REAL_LATE_DELIVERY

100%|████████████████████████████████████████████████████████████████████████████████| 91/91 [00:00<00:00, 4135.41it/s]


,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,543992763,2021-09-01 00:04:26,65RUZmnykIRzyWaqnOaB_EdQ,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,72.99,...,NaN,Chennai,72.99,True,NaN,False,6.92,False,1,2
1,545335809,2021-09-02 19:14:44,7xXdPBvy6-RXyGx3v7zyaSCA,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,203.92,...,NaN,Chennai,203.92,True,NaN,False,8.43,False,3,2
2,546603375,2021-09-04 11:48:23,2O7-Z2JeLESweHKSJ90ZWSPw,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,62.10,...,NaN,Delhi,62.10,True,NaN,False,5.11,False,1,1
3,547902466,2021-09-05 22:34:42,OHNkUE22wDR72kWu2FRPE7pw,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,197.84,...,NaN,Chennai,197.84,True,NaN,False,5.68,False,3,1
4,547981628,2021-09-06 02:00:46,1qqu5xqqphQqqd37qWbz0f0w,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,223.46,...,NaN,Chennai,223.46,True,NaN,False,5.78,False,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,642181463,2021-12-31 15:36:13,8C6wpZBWUMRXWYwbBL_7_IRw,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,480.50,...,NaN,Chennai,480.50,True,NaN,False,8.23,False,6,1
87,642216440,2021-12-31 16:20:47,VjJ5Rd2CXkQmCyGd2WENQ_EA,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,270.99,...,NaN,Chennai,270.99,True,NaN,False,5.81,False,3,1
88,643055740,2022-01-01 16:07:21,Tt1GikWeGxQXe-YwWtf0DQYA,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,484.00,...,NaN,Chennai,484.00,True,NaN,False,5.33,False,6,2
89,643444387,2022-01-01 23:06:56,T0JHCJSucwRXu3QtS7SKU2aA,False,False,LATE_DELIVERY,NaN,TRANSPORT,TGO,68.39,...,NaN,Chennai,103.99,False,NaN,True,NaN,False,1,1


In [13]:
# bad_reason : NO_COURIER
for i in tqdm(range(NO_COURIER.shape[0])) : 
    NO_COURIER.loc[NO_COURIER.index==i, 'satisfaction'] = satisfaction_score[random.choices(range(0, 5), weights=bad_reason_faild_weight)[0]]
NO_COURIER = NO_COURIER.assign(satisfaction=NO_COURIER['satisfaction'].astype(int))
NO_COURIER

100%|██████████████████████████████████████████████████████████████████████████████| 326/326 [00:00<00:00, 5014.18it/s]


,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,544013858,2021-09-01 01:00:45,Dt-SbfqG3dSbG6KlqdxMK8nQ,False,False,NO_COURIER,NaN,TRANSPORT,TGO,235.3800,...,NaN,Chennai,235.3800,True,NaN,False,6.16,False,3,2
1,544118387,2021-09-01 09:40:50,u7SALbJCsPTHCoKLJSw-Lm7g,False,False,NO_COURIER,NaN,TRANSPORT,TGO,67.8700,...,NaN,Chennai,67.8700,True,NaN,False,5.37,False,1,1
2,544183836,2021-09-01 11:46:51,iJC3hoYChSSeCOlYYR3o5nMA,False,False,NO_COURIER,NaN,TRANSPORT,TGO,59.9900,...,NaN,Hyderabad,59.9900,True,NaN,False,6.89,False,1,1
3,544746568,2021-09-01 23:31:30,1h4VLECO1HROO4aDCxRWFBSg,False,False,NO_COURIER,NaN,TRANSPORT,TGO,142.9900,...,NaN,Chennai,142.9900,True,NaN,False,9.39,False,2,1
4,544756605,2021-09-01 23:53:34,xdTkypTWs7S2WMwTTeetRXjA,False,False,NO_COURIER,NaN,TRANSPORT,TGO,147.9900,...,NaN,Chennai,147.9900,True,NaN,False,8.78,False,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,642498621,2021-12-31 20:37:09,1ixMkQNmOWT2mMCMNtDsT31g,False,False,NO_COURIER,NaN,TRANSPORT,TGO,175.9900,...,NaN,Delhi,175.9900,True,NaN,False,7.56,False,2,2
322,642502365,2021-12-31 20:40:52,rndvBNCiCmTUixMuCJn2e1Hw,False,False,NO_COURIER,NaN,TRANSPORT,TGO,159.9900,...,NaN,Chennai,209.9900,False,NaN,False,7.26,False,3,3
323,642684205,2022-01-01 01:12:55,TI6I_ndKjCSrK1cJdg2kkA_A,False,False,NO_COURIER,NaN,TRANSPORT,TGO,145.9942,...,NaN,Chennai,145.9942,True,NaN,False,6.96,False,2,2
324,642828634,2022-01-01 11:18:57,7xbx0ANCtTTDCUr_NuY_xNIA,False,False,NO_COURIER,NaN,TRANSPORT,TGO,224.9900,...,NaN,Delhi,299.9900,False,NaN,True,NaN,False,4,1


## Ressurecd

In [14]:
for i in tqdm(range(zomato_fail_ressurated.shape[0])) : 
    zomato_fail_ressurated.loc[zomato_fail_ressurated.index==i, 'satisfaction'] = satisfaction_score[random.choices(range(0, 5), weights=gold_success_weight)[0]]
zomato_fail_ressurated = zomato_fail_ressurated.assign(satisfaction=zomato_fail_ressurated['satisfaction'].astype(int))
zomato_fail_ressurated

100%|████████████████████████████████████████████████████████████████████████████| 6534/6534 [00:01<00:00, 4967.81it/s]


,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,544003009,2021-09-01 00:29:15,aercO_SKOCTaK1DdSAHzcRwg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TMP,235.00,...,NaN,Chennai,235.00,True,NaN,False,7.27,True,3,2
1,544035841,2021-09-01 02:38:50,iTjnN6CCYiTkCtoFCGkUh9Tg,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TMP,139.48,...,NaN,Delhi,139.48,True,NaN,False,6.49,True,2,3
2,544042740,2021-09-01 03:31:11,PFSLF6bmoLTUmlWQbjuaWC7A,False,False,NO_COURIER,NaN,VENDOR,TMP,90.00,...,NaN,Chennai,90.00,True,NaN,False,6.83,True,1,5
3,544146996,2021-09-01 10:44:38,TQCx8IzyX7Tgyt3azQHuY3wQ,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,39.19,...,NaN,Chennai,39.19,True,NaN,False,6.18,True,1,3
4,544164308,2021-09-01 11:15:43,w10UJYEexxS4eXraEqa9iJgQ,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,112.10,...,NaN,Chennai,112.10,True,NaN,False,5.02,True,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6529,644298082,2022-01-02 23:22:30,gGVPQfimlaSomX1oidrWYVpg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,70.99,...,NaN,Delhi,70.99,True,NaN,True,NaN,True,1,3
6530,644300790,2022-01-02 23:27:28,T3R-gTCSbiS2SPlJCXbrs_HA,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TMP,229.02,...,NaN,Chennai,229.02,True,NaN,False,6.22,True,3,5
6531,644302401,2022-01-02 23:30:12,zMdSeVGGvOSaGOEsGuuh0a5Q,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TMP,505.00,...,NaN,Chennai,505.00,True,NaN,True,NaN,True,6,4
6532,644304881,2022-01-02 23:35:13,yn81_8Q-pJTQ--7gQtrOjPvQ,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TGO,113.99,...,NaN,Mumbai,113.99,True,NaN,False,5.74,True,1,5


In [15]:
zomato_satisfaction = pd.concat([zomato_gold_success, zomato_regular_success, 
                                 ELSE_REASON, NOT_REAL_LATE_DELIVERY, 
                                 REAL_LATE_DELIVERY, NO_COURIER, 
                                 zomato_fail_ressurated], axis=0).sort_values('order_time').reset_index(drop=True)
zomato_satisfaction

,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected,portion,satisfaction
0,543991243,2021-09-01 00:00:51,ygyH_eaSHCTSSgAda33qDLYA,False,True,NaN,NaN,NaN,TGO,67.43,...,13.486,Chennai,53.944,False,20.0,False,5.44,False,1,4
1,543992115,2021-09-01 00:02:50,vagdCBP6L9S461DAPk0tNs-A,False,True,NaN,NaN,NaN,TGO,53.50,...,NaN,Mumbai,53.500,True,NaN,False,9.17,False,1,4
2,543992763,2021-09-01 00:04:26,65RUZmnykIRzyWaqnOaB_EdQ,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,72.99,...,NaN,Chennai,72.990,True,NaN,False,6.92,False,1,2
3,543993755,2021-09-01 00:06:41,THNssTsOurRqOJf9sQEhwiRg,False,True,NaN,NaN,NaN,TGO,206.00,...,NaN,Chennai,206.000,True,NaN,False,5.83,False,3,5
4,543994010,2021-09-01 00:07:13,OGX49Bsq0pR5qB32sT83JhSg,False,True,NaN,NaN,NaN,TGO,100.99,...,NaN,Chennai,100.990,True,NaN,False,7.41,False,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118195,644315564,2022-01-02 23:57:08,xdSbyNHiYORwiK2AHYieL5UQ,False,False,CUSTOMER_MISTAKE_ERROR,Changed mind - Mistake / Error,CUSTOMER,TGO,147.49,...,NaN,Chennai,147.490,True,NaN,False,6.62,False,2,1
118196,644315682,2022-01-02 23:57:25,AYeFPGmOUyT4OIWfmTSCYikw,False,True,NaN,NaN,NaN,TGO,110.49,...,NaN,Coimbatore,110.490,True,NaN,False,9.90,False,1,5
118197,644316048,2022-01-02 23:58:05,COnO0tda5iSTa-t5dFFVTpwg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,112.49,...,NaN,Chennai,112.490,True,NaN,False,7.36,False,1,2
118198,644316064,2022-01-02 23:58:07,iI6rRPz2LkRi2Ct0zTaksrOA,False,True,NaN,NaN,NaN,TGO,739.24,...,NaN,Chennai,739.240,True,NaN,False,5.17,False,10,2


In [16]:
zomato_satisfaction.to_csv('./data/zomato_preprocessed_4.csv', index=False)